In [ ]:
import sys
import os

# Add the parent directory to the Python path
parent_dir = os.path.abspath('../..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from resynthesis import resynthesis_from_model
import torch
import librosa
from IPython.display import Audio, display
import soundfile as sf

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# audio_path           = "evaluation_sounds/shards.wav"
# model_folder_path    = "checkpoints/model_261/"
# input_audio_improver = 1   # from 0 to 1. Enhance frequencies near the spectral centroid before the resynthesis for a cleaner representation
# energy_imposition    = 1   # eiter 0 to 1. Impose either the total energy or the energy per band of the input signal in a frame-by-frame basis either 
# envelope_follower    = 0.5 # from 0 to 1. Apply an envelope follower to the resynthesis. 0 = no envelope follower, 1 = sharp envelope follower. Numbers in between allowed.

# original    = torch.tensor(librosa.load(audio_path, sr=44100, mono=True)[0], device=device)
# resynthesis = resynthesis_from_model(audio_path          = audio_path,
#                                     model_folder_path    = model_folder_path,
#                                     input_audio_improver = input_audio_improver,
#                                     energy_imposition    = energy_imposition, 
#                                     envelope_follower    = envelope_follower,
#                                     print_parameters     = True)

# # Make them sound!
# from IPython.display import Audio
# display(Audio(original.detach().cpu().numpy(), rate=44100, normalize=True))
# display(Audio(resynthesis.detach().cpu().numpy(), rate=44100, normalize=True))

In [ ]:
synthesis_parameters = {
    "bubbles"  : [0.25, 1, 0.25],
    "fire"     : [0,    1, 0],
    "keyboard" : [0.25, 1, 0.25],
    "rain"     : [0,    1, 0.5],
    "river"    : [0.25, 1, 0.5],
    "shards"   : [0,    1, 0.5],
    "waterfall": [0.25, 1, 1],
    "wind"     : [1,    1, 0.25]
}

def full_synthesis(texture_type, input_audio_improver, energy_imposition, envelope_follower):
    model_paths = "checkpoints/"
    sound_paths = "evaluation_sounds_texDSP/"

    audio_path = os.path.join(sound_paths, f"{texture_type}.wav")
    original   = librosa.load(audio_path, sr=44100, mono=True)[0]
    print(f"Original sound: {texture_type}")
    display(Audio(original, rate=44100, normalize=True))
    # Make a list of all folders in model_paths that start with texture_type
    model_folders = [f for f in os.listdir(model_paths) if f.startswith(texture_type)]
    model_folders.sort()
    for model_folder in model_folders:
        model_folder_path = os.path.join(model_paths, model_folder)
        print(f"Processing {audio_path} with model {model_folder_path}")
        resynthesis = resynthesis_from_model(audio_path, model_folder_path, input_audio_improver, energy_imposition, envelope_follower).detach().cpu().numpy()
        sf.write(model_folder+'.wav', resynthesis, 44100)

texture_types = synthesis_parameters.keys()
# Iterate over all texture types
for texture_type in texture_types:
    # Get the synthesis parameters for the current texture type
    input_audio_improver, energy_imposition, envelope_follower = synthesis_parameters[texture_type]
    # Perform the full synthesis
    full_synthesis(texture_type, input_audio_improver, energy_imposition, envelope_follower)